# BERT input modify work

Derived from bert_classifiction.ipynb.

Modify to use model trained by colab.

In [1]:
!git clone -b docker https://github.com/karino2/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 170.15 KiB | 0 bytes/s, done.
Resolving deltas: 100% (155/155), done.
Checking connectivity... done.


In [2]:
!ls bert/

CONTRIBUTING.md		    modeling.py		  run_pretraining.py
Dockerfile		    modeling_test.py	  run_squad.py
LICENSE			    multilingual.md	  sample_text.txt
README.md		    optimization.py	  tokenization.py
__init__.py		    optimization_test.py  tokenization_test.py
create_pretraining_data.py  requirements.txt	  utils
extract_features.py	    run_classifier.py


In [1]:
!pip3 install -r ./bert/requirements.txt

    100% |################################| 83.1MB 15kB/s  eta 0:00:01  6% |##                              | 5.6MB 43.9MB/s eta 0:00:02    30% |#########                       | 25.6MB 44.7MB/s eta 0:00:02    57% |##################              | 47.6MB 49.0MB/s eta 0:00:01    60% |###################             | 49.9MB 46.2MB/s eta 0:00:01    74% |#######################         | 62.2MB 30.5MB/s eta 0:00:01    76% |########################        | 63.7MB 34.3MB/s eta 0:00:01
    100% |################################| 10.1MB 134kB/s eta 0:00:01   11% |###                             | 1.1MB 55.8MB/s eta 0:00:01    84% |##########################      | 8.5MB 50.7MB/s eta 0:00:01
    100% |################################| 3.1MB 449kB/s eta 0:00:01
    100% |################################| 102kB 11.1MB/s a 0:00:01
    100% |################################| 51kB 9.8MB/s eta 0:00:01
    100% |################################| 13.8MB 101kB/s eta 0:00:01   11% |###                

### Model and data download

We solve RTE task in GLUE datasets; see https://www.nyu.edu/projects/bowman/glue.pdf in detail.

In [1]:
import os

In [5]:
os.makedirs("./bert/model", exist_ok=True)
os.makedirs("./bert/data", exist_ok=True)

In [6]:
# run this from command line.

!gsutil cp -r gs://yohei-kikuta/mlstudy-phys/bert/data/1000 ./bert/data/

/bin/sh: 1: gsutil: not found


In [19]:
# Run this outside of container.
# need vocab, etc...(may be enough just copy those files, but just copy everthing)

!gsutil cp gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/vocab.txt ./bert/model/patent/vocab.txt
!gsutil cp gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_config.json ./bert/model/bert_config.json

/bin/sh: 1: gsutil: not found


In [ ]:
! gsutil cp -r gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000/model.ckpt-1602* ./bert/model/patent_1000

In [30]:
### try for head 100 data only
# head -n 100 1000/dev.tsv > 100/dev.tsv

### Load dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv("./bert/data/1000/train.tsv", sep="\t")
test_data = pd.read_csv("./bert/data/1000/dev.tsv", sep="\t")

In [3]:
test_data.head()

,index,claim_app,claim_cited_grant,label
0,0,1 . A rotational angle detection device for a ...,1. A tangible computer-readable medium having ...,not_entailment
1,1,1 . A method comprising:sensing electrocardiog...,"1. A medical system, comprising:an implantable...",entailment
2,2,1 . A system for controlling temperature insid...,1. An integrated differential receiver for an ...,not_entailment
3,3,1 . A recording apparatus for recording update...,1. A method for indicating a priority of a Voi...,not_entailment
4,4,1 . Apparatus for moving a material within an ...,1. A wellbore packer having an expandable pack...,entailment


## Inference using a trained BERT model.

In [4]:
import sys
sys.path.append("./bert")

In [5]:
import tensorflow as tf

In [6]:
# TPUEstimator do not support eager execution!
# tf.enable_eager_execution()

In [7]:
import os

import modeling
import optimization
import tokenization

from run_classifier import RteProcessor
from run_classifier import model_fn_builder
from run_classifier import file_based_input_fn_builder
from run_classifier import file_based_convert_examples_to_features

### Predict all test data from the test.tsv

In [8]:
bert_config = modeling.BertConfig.from_json_file("./bert/model/patent/bert_config.json")

In [9]:
class FLAGS(object):
    '''Parameters.'''
    def __init__(self):
        self.vocab_file = "./bert/model/patent/vocab.txt"
        self.do_lower_case = True
        self.use_tpu = False
        self.output_dir = "./bert/data/bert-training-results/patent_1000"
        self.data_dir = "./bert/data/100"
        self.max_seq_length = 512
        self.init_checkpoint = "./bert/model/patent_1000/model.ckpt-1602"
        self.predict_batch_size = 16
        
        # The following parameters are not used in predictions.
        # Just use to create RunConfig.
        self.master = None
        self.save_checkpoints_steps = 1
        self.iterations_per_loop = 1
        self.num_tpu_cores = 1
        self.learning_rate = 0
        self.num_warmup_steps = 0
        self.num_train_steps = 0
        self.train_batch_size = 0
        self.eval_batch_size = 0

In [10]:
FLAGS = FLAGS()

In [11]:
tokenizer = tokenization.FullTokenizer(
    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

tpu_cluster_resolver = None

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

In [12]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(["not_entailment", "entailment"]),
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=FLAGS.num_train_steps,
    num_warmup_steps=FLAGS.num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_tpu)


estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_train_distribute': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_cluster': None, '_tf_random_seed': None, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_evaluation_master': '', '_is_chief': True, '_tpu_config': TPUConfig(iterations_per_loop=1, num_shards=1, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_save_summary_steps': 100, '_num_worker_replicas': 1, '_global_id_in_cluster': 0, '_model_dir': './bert/data/bert-training-results/patent_1000', '_master': '', '_save_checkpoints_steps': 1, '_experimental_distribute': None, '_protocol': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fede8be1898>, '_log_step_count_steps': None, '_device_fn

In [13]:
import collections

In [14]:
from run_classifier import convert_single_example
from run_classifier import InputExample
from run_classifier import file_based_input_fn_builder

In [15]:
label = test_data['label']

In [16]:
MAX_SEQ_LEN = 512

In [17]:
def row_to_feature(i, row, max_seq_len=MAX_SEQ_LEN):
    if 'label' in row.index:
        example = InputExample(
            guid = row['index'],
            text_a = row['claim_app'],
            text_b = row['claim_cited_grant'],
            label = row['label']
            )
    else :
        example = InputExample(
            guid = row['index'],
            text_a = row['claim_app'],
            text_b = row['claim_cited_grant'],
            )
    
    return convert_single_example(
        ex_index=i, 
        example=example,
        label_list=["not_entailment", "entailment"],
        max_seq_length=max_seq_len,
        tokenizer=tokenizer)

In [19]:
def feature_list_to_feature_dict(flist):
    dic = {}
    dic['input_ids'] = [feature.input_ids for feature in flist]
    dic["input_mask"] = [feature.input_mask for feature in flist]
    dic["segment_ids"] = [feature.segment_ids for feature in flist]
    dic["label_ids"] = [[feature.label_id] for feature in flist]
    return dic

In [43]:
def df_to_fdicts(df):
    features = [row_to_feature(i, row) for i, row in  df.iterrows()]
    return feature_list_to_feature_dict(features)

### The cell below takes a few minutes.

In [44]:
%%time

test_fdict_all = df_to_fdicts(test_data)

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 0
INFO:tensorflow:tokens: [CLS] 1 . a rotational angle detection device for a permanent magnet dynamo - electric machine comprising : a stat ##or that includes arm ##at ##ure winding ; a rotor that includes a rotor core and a plurality of permanent magnet ##s ; and a conduct ##ive circuit that includes first conductors extending in an axial direction of the rotor and being disposed in at least two places in a ci ##rc ##um ##fer ##ential direction of the rotor , and second conductors for electrically inter ##con ##ne ##cting the first conductors , wherein , the first conductors are each disposed within one of a range of an electrical angle between & # x ##22 ##12 ; 45 & # x ##b ##0 ; and + 45 & # x ##b ##0 ; in the ci ##rc ##um ##fer ##ential direction with a magnetic pole center of each of the plurality of permanent magnet ##s being set as a reference and a range of an electrical angle between & # x ##22 ##12 ; 45 & # x ##b ##0 ; an

INFO:tensorflow:input_ids: 101 1015 1012 1037 4118 9605 1024 13851 16175 11522 3695 13113 1006 14925 2290 1007 2951 2012 2019 14925 2290 13617 2291 1025 6364 1996 10596 14925 2290 2951 2012 1996 14925 2290 13617 2291 2000 5646 2028 2030 2062 5005 5300 1010 16726 2169 5005 3643 2003 24668 1997 1037 10903 1997 1037 6922 1997 1996 10596 14925 2290 2951 2008 2003 9280 3141 2000 1037 2512 1011 2540 3786 3120 1025 1998 6016 1996 2028 2030 2062 5005 5300 2000 2019 12978 13460 23416 2075 2291 2000 13460 23416 1996 14925 2290 13617 2291 1012 1016 1012 1996 4118 1997 4366 1015 1010 16726 1996 12978 13460 23416 2075 2291 10438 2019 4106 1997 1996 2028 2030 2062 5005 5300 1998 14847 1037 3120 1997 1037 3327 6922 1997 1996 10596 14925 2290 2951 2008 2003 3141 2000 1037 3327 2512 1011 2540 3786 3120 2241 2006 1996 4106 1012 1017 1012 1996 4118 1997 4366 1016 1010 2582 9605 1010 2044 6016 1996 2028 2030 2062 5005 5300 1010 4909 2019 7953 1998 1010 26651 2000 1996 7953 1010 26709 6593 17441 2028 2030 

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

INFO:tensorflow:label: entailment (id = 1)
CPU times: user 1min 58s, sys: 12 ms, total: 1min 58s
Wall time: 1min 58s


In [18]:
features = [row_to_feature(i, row) for i, row in  test_data.iterrows()]

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 0
INFO:tensorflow:tokens: [CLS] 1 . a rotational angle detection device for a permanent magnet dynamo - electric machine comprising : a stat ##or that includes arm ##at ##ure winding ; a rotor that includes a rotor core and a plurality of permanent magnet ##s ; and a conduct ##ive circuit that includes first conductors extending in an axial direction of the rotor and being disposed in at least two places in a ci ##rc ##um ##fer ##ential direction of the rotor , and second conductors for electrically inter ##con ##ne ##cting the first conductors , wherein , the first conductors are each disposed within one of a range of an electrical angle between & # x ##22 ##12 ; 45 & # x ##b ##0 ; and + 45 & # x ##b ##0 ; in the ci ##rc ##um ##fer ##ential direction with a magnetic pole center of each of the plurality of permanent magnet ##s being set as a reference and a range of an electrical angle between & # x ##22 ##12 ; 45 & # x ##b ##0 ; an

INFO:tensorflow:input_ids: 101 1015 1012 1037 4118 9605 1024 13851 16175 11522 3695 13113 1006 14925 2290 1007 2951 2012 2019 14925 2290 13617 2291 1025 6364 1996 10596 14925 2290 2951 2012 1996 14925 2290 13617 2291 2000 5646 2028 2030 2062 5005 5300 1010 16726 2169 5005 3643 2003 24668 1997 1037 10903 1997 1037 6922 1997 1996 10596 14925 2290 2951 2008 2003 9280 3141 2000 1037 2512 1011 2540 3786 3120 1025 1998 6016 1996 2028 2030 2062 5005 5300 2000 2019 12978 13460 23416 2075 2291 2000 13460 23416 1996 14925 2290 13617 2291 1012 1016 1012 1996 4118 1997 4366 1015 1010 16726 1996 12978 13460 23416 2075 2291 10438 2019 4106 1997 1996 2028 2030 2062 5005 5300 1998 14847 1037 3120 1997 1037 3327 6922 1997 1996 10596 14925 2290 2951 2008 2003 3141 2000 1037 3327 2512 1011 2540 3786 3120 2241 2006 1996 4106 1012 1017 1012 1996 4118 1997 4366 1016 1010 2582 9605 1010 2044 6016 1996 2028 2030 2062 5005 5300 1010 4909 2019 7953 1998 1010 26651 2000 1996 7953 1010 26709 6593 17441 2028 2030 

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

INFO:tensorflow:label: entailment (id = 1)


In [46]:
len(features)

2502

In [47]:
test_fdict_all = feature_list_to_feature_dict(features)

In [48]:
def peek_fdict_list(fdict_list, count):
    res_fdict = {}
    for key, val in fdict_list.items():
        res_fdict[key] = val[0:count]
    return res_fdict

In [53]:
test_fdict_100 = peek_fdict_list(test_fdict_all, 100)

In [54]:
test_fdict = test_fdict_100

In [55]:
def batch_input_function(params):
    test_ds = tf.data.Dataset.from_tensor_slices(test_fdict)    
    return test_ds.batch(params['batch_size'])

In [27]:
result = estimator.predict(batch_input_function)

In [28]:
%%time
#It will take about one hour on a CPU environment.

result = list(result)

INFO:tensorflow:Could not find trained model in model_dir: ./bert/data/bert-training-results/patent_1000, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = label_ids, shape = (?, 1)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tenso

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FRO

INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert

In [29]:
len(result)

100

#### Label { 0: not cited, 1: cited }

In [36]:
test_100 = test_data[0:100].copy()

In [37]:
test_100['label'] = [ 0 if elem == 'not_entailment' else 1 for elem in test_100['label'] ]

In [38]:
test_100.head()

,index,claim_app,claim_cited_grant,label
0,0,1 . A rotational angle detection device for a ...,1. A tangible computer-readable medium having ...,0
1,1,1 . A method comprising:sensing electrocardiog...,"1. A medical system, comprising:an implantable...",1
2,2,1 . A system for controlling temperature insid...,1. An integrated differential receiver for an ...,0
3,3,1 . A recording apparatus for recording update...,1. A method for indicating a priority of a Voi...,0
4,4,1 . Apparatus for moving a material within an ...,1. A wellbore packer having an expandable pack...,1


In [39]:
test_100['prediction_score'] = result

In [42]:
test_100.head()

,index,claim_app,claim_cited_grant,label,prediction_score,prediction_label
0,0,1 . A rotational angle detection device for a ...,1. A tangible computer-readable medium having ...,0,"[0.9998567, 0.00014324456]",0
1,1,1 . A method comprising:sensing electrocardiog...,"1. A medical system, comprising:an implantable...",1,"[0.00012639465, 0.99987364]",1
2,2,1 . A system for controlling temperature insid...,1. An integrated differential receiver for an ...,0,"[0.99986386, 0.00013612612]",0
3,3,1 . A recording apparatus for recording update...,1. A method for indicating a priority of a Voi...,0,"[0.9998641, 0.00013589088]",0
4,4,1 . Apparatus for moving a material within an ...,1. A wellbore packer having an expandable pack...,1,"[0.00010087702, 0.99989915]",1


In [40]:
test_100['prediction_label'] = [
    1 if score_1 >= 0.5 else 0 for score_0, score_1 in test_100['prediction_score']
]

In [41]:
test_100.head()

,index,claim_app,claim_cited_grant,label,prediction_score,prediction_label
0,0,1 . A rotational angle detection device for a ...,1. A tangible computer-readable medium having ...,0,"[0.9998567, 0.00014324456]",0
1,1,1 . A method comprising:sensing electrocardiog...,"1. A medical system, comprising:an implantable...",1,"[0.00012639465, 0.99987364]",1
2,2,1 . A system for controlling temperature insid...,1. An integrated differential receiver for an ...,0,"[0.99986386, 0.00013612612]",0
3,3,1 . A recording apparatus for recording update...,1. A method for indicating a priority of a Voi...,0,"[0.9998641, 0.00013589088]",0
4,4,1 . Apparatus for moving a material within an ...,1. A wellbore packer having an expandable pack...,1,"[0.00010087702, 0.99989915]",1


# == Try and Error =====

### Evaluate 100 dev set

In [51]:
%%time

!python3 ./bert/run_classifier.py \
  --task_name=RTE \
  --do_train=false \
  --do_eval=true \
  --data_dir=./bert/data/100 \
  --vocab_file=./bert/model/patent/vocab.txt \
  --bert_config_file=./bert/model/patent/bert_config.json \
  --init_checkpoint=./bert/model/patent_1000/model.ckpt-1602 \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir=./bert/tmp/rte_output/

INFO:tensorflow:Using config: {'_service': None, '_save_checkpoints_secs': None, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_train_distribute': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_task_id': 0, '_eval_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f47c93e4518>, '_num_ps_replicas': 0, '_task_type': 'worker', '_cluster': None, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_log_step_count_steps': None, '_global_id_in_cluster': 0, '_is_chief': True, '_experimental_distribute': None, '_model_dir': './bert/tmp/rte_output/', '_save_summary_steps': 100, '_num_work

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: dev-4
INFO:tensorflow:tokens: [CLS] 1 . apparatus for moving a material within an enclosure comprising : a field - responsive fluid including a plurality of particles suspended in a base fluid , the field - responsive fluid disposed in the enclosure ; a field generating source which generates a field in response to which at least some of the plurality of particles align , the field being [SEP] 1 . a well ##bor ##e pack ##er having an expand ##able packing element for sealing a well ann ##ulus , an act ##ua ##tor for expanding said packing element into operative engagement across said ann ##ulus and an electrically control ##lab ##le fluid for controlling the operation of said act ##ua ##tor , wherein said electrically control ##lab ##le fluid is [SEP]
INFO:tensorflow:input_ids: 101 1015 1012 14709 2005 3048 1037 3430 2306 2019 17539 9605 1024 1037 2492 1011 26651 8331 2164 1037 29018 1997 9309 6731 1999 1037 2918 8331 1010 1996 2492

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-19-09:57:33
INFO:tensorflow:Graph was finalized.
2018-12-19 09:57:34.145835: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-12-19 09:57:35.100315: W tensorflow/core/framework/allocator.cc:122] Allocation of 93763584 exceeds 10% of system memory.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-19-09:59:33
INFO:tensorflow:Saving dict for global step 0: eval_accuracy = 0.9191919, eval_loss = 0.45242003, global_step = 0, loss = 0.43067768
INFO:tensorflow:evaluation_loop marked as finished
INFO:tensorflow:***** Eval results *****
INFO:tensorflow:  eval_accuracy = 0.9191919
INFO:tensorflow:  eval_loss = 0.45242003
INFO:tensorflow:  global_step = 0
INFO:tensorflow:  loss = 0.43067768
CPU times: user 2.44 s, sys: 360 ms, total

In [140]:
accuracy = len( test_df[test_df['label'] == test_df['prediction_label']] ) / len( test_df['label'] )
precision = len( test_df[(test_df['label'] == 1) & (test_df['prediction_label'] == 1)] ) / sum( test_df['label'] == 1 )
recall = len( test_df[(test_df['label'] == 1) & (test_df['prediction_label'] == 1)] ) / sum( test_df['prediction_label'] == 1 )

In [143]:
print(accuracy)
print(precision)
print(recall)

0.9372501998401279
0.9232613908872902
0.9498355263157895
